In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map


In [3]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(URL)

In [4]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [5]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [8]:
for _, row in launch_sites_df.iterrows():
    # Add a circle around the launch site
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=50,  # Radius in meters
        color='#d35400',  # Orange color
        fill=True,
        fill_color='#d35400',
        fill_opacity=0.3,
        popup=row['Launch Site']
    ).add_to(site_map)

    # Add a marker at the launch site
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(150,36),  # Adjust size to fit the text
            icon_anchor=(7,20),  # Adjust position of the text
            html='<div style="font-size: 12pt; color:#d35400;"><b>{}</b></div>'.format(row['Launch Site'])
        )
    ).add_to(site_map)
site_map.save('launch_sites_map.html')

In [9]:
marker_cluster = MarkerCluster()

# Task 2: Mark the success/failed launches for each site on the map

In [10]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

In [11]:
site_map.add_child(marker_cluster)
for index, record in spacex_df.iterrows():
    # Extract the latitude, longitude, and launch success status
    latitude = record['Lat']
    longitude = record['Long']
    
    # Create a Marker with customized icon
    marker = folium.Marker(
        location=[latitude, longitude],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    
    # Add the marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Add the MarkerCluster to the map
site_map.add_child(marker_cluster)

# Save or display the map
site_map.save('map.html')

In [12]:
site_map

# TASK 3: Calculate the distances between a launch site to its proximities


In [13]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

In [14]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [15]:
# find coordinate of the closet coastline
coastline_lat = 28.56257  
coastline_lon = -80.56787
launch_site_lat = 28.563197
launch_site_lon = -80.576820
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [16]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_marker = folium.Marker(
   (launch_site_lat, launch_site_lon),
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   )

In [17]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
coastline_points = [(coastline_lat, coastline_lon)]
coordinates = [(launch_site_lat, launch_site_lon)] + coastline_points
lines = folium.PolyLine(locations=coordinates, weight=1, color='blue')
site_map.add_child(lines)

#### Railway

In [20]:
# find coordinate of the closet coastline
railway_lat = 28.57122
railway_lon = -80.58538
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

In [21]:
distance_marker = folium.Marker(
   (launch_site_lat, launch_site_lon),
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
       )
   )

In [22]:
coastline_points = [(railway_lat, railway_lon)]
coordinates = [(launch_site_lat, launch_site_lon)] + coastline_points
lines_railway = folium.PolyLine(locations=coordinates, weight=1, color='blue')
site_map.add_child(lines_railway)

In [23]:
# find coordinate of the closet coastline
highway_lat = 28.56382
highway_lon = -80.57083
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

In [24]:
distance_marker = folium.Marker(
   (launch_site_lat, launch_site_lon),
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
       )
   )

In [25]:
coastline_points = [(highway_lat, highway_lon)]
coordinates = [(launch_site_lat, launch_site_lon)] + coastline_points
lines_highway = folium.PolyLine(locations=coordinates, weight=1, color='blue')
site_map.add_child(lines_highway)

In [29]:
# find coordinate of the closet coastline
city_lat = 28.61333
city_lon = -80.81027
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

In [30]:
distance_marker = folium.Marker(
   (launch_site_lat, launch_site_lon),
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
       )
   )

In [31]:
coastline_points = [(city_lat, city_lon)]
coordinates = [(launch_site_lat, launch_site_lon)] + coastline_points
lines_city = folium.PolyLine(locations=coordinates, weight=1, color='blue')
site_map.add_child(lines_city)

In [18]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [19]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance